### Getting the data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Q2. Indexing the data

In [4]:
from elasticsearch import Elasticsearch 

es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '93a31832c6b4', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'RG8Nb1QfSHeObw4KXaVtIA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-1-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-1-questions'})

In [6]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:19<00:00, 49.41it/s]


### Q3. Searching

In [7]:
query = "How do I execute a command in a running docker container?"

In [8]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [10]:
top_score = response['hits']['hits'][0]['_score']
print(f"Top ranking result score:{top_score}")

Top ranking result score:84.050095


### Q4. Filtering

In [13]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [15]:
result_docs = []

for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

In [16]:
result_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Q5. Building a prompt 

In [17]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [22]:
context = []

for document in result_docs:
    context.append(context_template.format(question=document['question'], text=document['text']))

context = "\n\n".join(context)

In [23]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [24]:
prompt = prompt_template.format(question = query, context= context)

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [25]:
len(prompt)

1462

### Q6. Tokens

In [26]:
import tiktoken

In [27]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [28]:
num_tokens = len(encoding.encode(prompt))
print(f"Number of tokens:{num_tokens}")

Number of tokens:322


In [30]:
encoding.decode_single_token_bytes(63842)

b"You're"

### Q7. Generating the answer

In [31]:
from openai import OpenAI

In [32]:
import os
os.environ['OPENAI_API_KEY'] = 'my-api-key'

In [33]:
client = OpenAI()

In [34]:
response = client.chat.completions.create(
    model = 'gpt-4o', 
    messages=[{"role":"user", "content":prompt}]
)

In [46]:
response_text = response.choices[0].message.content.strip()

print(response_text)

To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps to do so:

1. List all running containers to find the container ID or name of the specific container you want to execute a command in:
   ```shell
   docker ps
   ```

2. Use the `docker exec` command along with the `-it` flags to run a command in the specified container. For example, to start a bash shell inside the container:
   ```shell
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID or name of the container.


### Q8. Calculating the costs

In [49]:
num_tokens_received = len(encoding.encode(response_text))

num_tokens_received

135

In [51]:
token_input = 0.005/1000

token_output = 0.015/1000

total_price = token_input*num_tokens + token_output*num_tokens_received

print(f"Number of prompt tokens: {num_tokens}")
print(f"Number of response tokens: {num_tokens_received}")
print(f"Total price: {total_price}$")

Number of prompt tokens: 322
Number of response tokens: 135
Total price: 0.003635$
